# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hithadhoo,-0.6000,73.0833,28.34,75,96,5.41,MV,1706942705
1,1,wetaskiwin,52.9668,-113.3687,1.12,90,100,2.46,CA,1706942705
2,2,kerikeri,-35.2268,173.9474,20.51,65,0,1.34,NZ,1706942705
3,3,vorkuta,67.5000,64.0000,-13.83,88,100,5.20,RU,1706942705
4,4,mahina,-17.5065,-149.4890,27.87,69,20,1.03,PF,1706942706


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

city_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.60,
    color= "City"
)
# YOUR CODE HERE

# Display the map
city_humidity_map

c:\Users\danrk\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[city_data_df["Humidity"] <= 66]
city_data_df = city_data_df.loc[city_data_df["Max Temp"] < 34]
city_data_df = city_data_df.loc[city_data_df["Max Temp"] >= 0]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,kerikeri,-35.2268,173.9474,20.51,65,0,1.34,NZ,1706942705
5,5,nguigmi,14.2495,13.1092,16.07,22,3,6.83,NE,1706942706
10,10,lihue,21.9789,-159.3672,21.58,58,75,12.35,US,1706942707
20,20,san sebastian de la gomera,28.0916,-17.1133,18.20,31,0,0.00,ES,1706942710
21,21,blackmans bay,-43.0167,147.3167,27.07,54,5,5.32,AU,1706942710
...,...,...,...,...,...,...,...,...,...,...
576,576,didwana,27.4000,74.5667,24.40,33,100,5.22,IN,1706942828
578,578,kos,36.8933,27.2889,11.69,64,87,6.09,GR,1706942828
583,583,igbor,7.4527,8.6098,23.70,22,3,4.03,NG,1706942830
584,584,guerrero negro,27.9769,-114.0611,15.32,62,0,7.25,MX,1706942830


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,kerikeri,NZ,-35.2268,173.9474,65,
5,nguigmi,NE,14.2495,13.1092,22,
10,lihue,US,21.9789,-159.3672,58,
20,san sebastian de la gomera,ES,28.0916,-17.1133,31,
21,blackmans bay,AU,-43.0167,147.3167,54,
...,...,...,...,...,...,...
576,didwana,IN,27.4000,74.5667,33,
578,kos,GR,36.8933,27.2889,64,
583,igbor,NG,7.4527,8.6098,22,
584,guerrero negro,MX,27.9769,-114.0611,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

params = {
    "categories" : categories,
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["limit"] = 5
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data

hotel_df

Starting hotel search
kerikeri - nearest hotel: Avalon Resort
nguigmi - nearest hotel: Guest PAM
lihue - nearest hotel: Kauai Palms
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera
blackmans bay - nearest hotel: Villa Howden
malvan - nearest hotel: Hotel Silver Sand
port lincoln - nearest hotel: Boston Hotel
bandarbeyla - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
saipan - nearest hotel: Chalan Kanoa Beach Hotel
tura - nearest hotel: No hotel found
dunedin - nearest hotel: Scenic Hotel Dunedin City
nazareno - nearest hotel: No hotel found
road town - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
kodinar - nearest hotel: No hotel found
kavaratti - nearest hotel: No hotel found
as sulayyil - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
bu'aale - nearest hotel: No hotel found
phan thiet - nearest hotel: Khách sạn Cà Ty
taichung - nearest hotel: 塔

,City,Country,Lat,Lng,Humidity,Hotel Name
2,kerikeri,NZ,-35.2268,173.9474,65,Avalon Resort
5,nguigmi,NE,14.2495,13.1092,22,Guest PAM
10,lihue,US,21.9789,-159.3672,58,Kauai Palms
20,san sebastian de la gomera,ES,28.0916,-17.1133,31,Hotel Villa Gomera
21,blackmans bay,AU,-43.0167,147.3167,54,Villa Howden
...,...,...,...,...,...,...
576,didwana,IN,27.4000,74.5667,33,No hotel found
578,kos,GR,36.8933,27.2889,64,MARITINA
583,igbor,NG,7.4527,8.6098,22,No hotel found
584,guerrero negro,MX,27.9769,-114.0611,62,Plaza sal paraiso


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
#%%capture --no-display

# Configure the map plot
vacation_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.80,
    color= "City",
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
vacation_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)